# Automatic Goggles - Example Usage

This notebook demonstrates the **automatic-goggles** package for extracting structured fields from conversation transcripts.

## Features Demonstrated:
1. Basic field extraction with reasoning
2. Field extraction without reasoning (faster, lower cost)
3. Complex multi-field extraction (names, contact info, dates, company info)
4. Conversation evaluation against custom criteria

## Step 1: Install the Package

First, install the `automatic-goggles` package from PyPI:

In [ ]:
!pip install --upgrade automatic-goggles

## Step 2: Import Required Libraries

Import the necessary modules:

In [ ]:
import json
import os
from transtype import TranscriptProcessor, AssertsEvaluator

## Step 3: Set Your OpenAI API Key

You'll need an OpenAI API key to use this package:

In [ ]:
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# Option 1: Load from environment variable (recommended)
api_key = os.getenv("OPENAI_API_KEY")

# Option 2: Set it directly (not recommended for production)
if not api_key:
    api_key = input("Enter your OpenAI API key: ").strip()

## Step 4: Initialize Processors

Create two processors - one with reasoning and one without:

In [ ]:
# Define fields for basic examples
basic_fields = [
    {
        "field_name": "representative_name",
        "field_type": "string",
        "format_example": "Sarah Chen",
        "field_description": "The name of the customer service representative or agent who is helping the customer. This should be extracted from their introduction or when they identify themselves during the conversation.",
    }
]

# With reasoning (provides explanations)
processor_with_reasoning = TranscriptProcessor(api_key=api_key, fields=basic_fields, include_reasoning=True)

# Without reasoning (faster, lower cost)
processor_without_reasoning = TranscriptProcessor(api_key=api_key, fields=basic_fields, include_reasoning=False)

print("✅ Processors initialized successfully!")


## Example 1: Basic Field Extraction (With Reasoning)

Extract a representative's name from a simple conversation:

In [ ]:
basic_input_data = {
    "messages": [
        {
            "role": "assistant",
            "content": "Hi, this is Marcus, I'm a customer service representative with TechFlow Solutions in Downtown Seattle. Our account manager is currently in a meeting, but I'm here to help with any questions. What can I assist you with today?",
        },
        {"role": "user", "content": "I need to discuss my account billing issues."},
        {
            "role": "assistant",
            "content": "I understand you have billing concerns. I can definitely help you with that. Let me pull up your account information. Can you please provide me with your account number or the email address associated with your account?",
        },
    ],
}

print("Processing basic example with reasoning...")
result_with_reasoning = processor_with_reasoning.process(basic_input_data)
print("\n✅ Extraction Result (with reasoning):")
print(json.dumps(result_with_reasoning, indent=2))


## Example 2: Basic Field Extraction (Without Reasoning)

Same extraction but without reasoning explanations (faster & cheaper):

In [ ]:
print("Processing basic example without reasoning...")
result_without_reasoning = processor_without_reasoning.process(basic_input_data)
print("\n✅ Extraction Result (without reasoning):")
print(json.dumps(result_without_reasoning, indent=2))

## Example 3: Complex Multi-Field Extraction

Extract multiple fields from an insurance claim conversation:

In [ ]:
# Define fields for complex example
complex_fields = [
    {
        "field_name": "agent_name",
        "field_type": "string",
        "format_example": "Jennifer Martinez",
        "field_description": "The full name of the insurance agent, customer service representative, or company employee who is assisting the customer. Look for introductions like 'This is [Name] from [Company]' or when they identify themselves during the conversation.",
    },
    {
        "field_name": "customer_name",
        "field_type": "string",
        "format_example": "Michael Torres",
        "field_description": "The customer's full name or last name. This could be mentioned when the agent addresses them (e.g., 'Mr. Torres', 'Ms. Johnson') or when the customer introduces themselves. Look for formal address patterns and name references.",
    },
    {
        "field_name": "customer_phone",
        "field_type": "string",
        "format_example": "415-555-9876",
        "field_description": "The customer's current phone number mentioned during the call. Pay attention to any updates or corrections to phone numbers during the conversation. Look for 10-digit numbers in formats like 415-555-9876, (415) 555-9876, or similar patterns.",
    },
    {
        "field_name": "customer_email",
        "field_type": "string",
        "format_example": "michael.torres@techcorp.com",
        "field_description": "The customer's email address provided or confirmed during the conversation. Look for standard email format with @ symbol and domain. This might be mentioned for verification or contact purposes.",
    },
    {
        "field_name": "incident_date",
        "field_type": "string",
        "format_example": "December 8, 2024",
        "field_description": "The exact date when the incident, accident, or event being discussed occurred. This is the base date for calculating other related dates. Look for dates mentioned as 'accident date', 'incident date', or when the event happened.",
    },
    {
        "field_name": "resolution_date",
        "field_type": "string",
        "format_example": "January 15, 2024",
        "field_description": "The expected or promised date when the issue, claim, or matter will be resolved or completed. This could be mentioned as 'resolved by', 'completed by', 'finalized by', or similar future date references related to case resolution.",
    },
    {
        "field_name": "appointment_date",
        "field_type": "string",
        "format_example": "December 13, 2024",
        "field_description": "The calculated appointment date that is always scheduled for 'next Friday' relative to the incident date. When the conversation mentions 'next week Friday after the incident date' or 'Friday of the following week from the accident', calculate the actual date. For incident on December 3, 2024 (Tuesday), next week Friday would be December 13, 2024.",
    },
    {
        "field_name": "company_name",
        "field_type": "string",
        "format_example": "AutoClaim Solutions",
        "field_description": "The name of the company, organization, or business that the agent represents. This is typically mentioned in the agent's introduction like 'calling from [Company Name]' or 'I'm with [Company Name]'.",
    },
]

complex_processor = TranscriptProcessor(api_key=api_key, fields=complex_fields, include_reasoning=True)

complex_input_data = {
    "messages": [
        {
            "role": "assistant",
            "content": "Good morning! This is Joan London from ABC Insurance Solutions. I'm calling regarding your recent vehicle damage claim. How are you today?",
        },
        {
            "role": "user",
            "content": "Hi Joan, I'm doing okay considering the circumstances. I submitted the claim for the accident that happened on August 11th, 2025.",
        },
        {
            "role": "assistant",
            "content": "I understand, and I'm sorry to hear about your accident. Let me pull up your claim details. Can you verify your contact information? I show your phone as 217-218-1234 and your email as will.tis@gmail.com. Is this still current?",
        },
        {
            "role": "user",
            "content": "Actually, my phone number has changed. It's now 217-918-2344, but the email address will.tis@gmail.com is correct.",
        },
        {
            "role": "assistant",
            "content": "Perfect, I've updated your phone to 217-918-2344. Thank you, Mr. Tesco. Now, regarding the resolution timeline, we expect to have everything finalized by January 15th, 2026. However, we do need to schedule an inspection appointment.",
        },
        {
            "role": "user",
            "content": "When would that inspection be? I hope it's soon so we can get this moving.",
        },
        {
            "role": "assistant",
            "content": "Absolutely! Since the accident was on August 11th, yesterday, we always schedule inspections for the Friday of the following week. So your appointment will be next week Friday from the incident date. Let me see... that would be perfect for our inspector's schedule.",
        },
        {
            "role": "user",
            "content": "So just to confirm - the appointment is the next week Friday after August 11 2025? And will this be with the same adjuster who handled my previous claim?",
        },
        {
            "role": "assistant",
            "content": "That's correct - next week Friday after the incident date. And no, this will be with our specialist David Chen, not the previous adjuster. He'll meet you at the collision center on Oak Avenue at 10:30 AM.",
        },
    ],
}

print("Processing complex multi-field example...")
complex_result = complex_processor.process(complex_input_data)
print("\n✅ Complex Extraction Result:")
print(json.dumps(complex_result, indent=2))


## Summary: Extraction Results

Let's display a formatted summary of all extracted fields:

In [ ]:
print("=" * 60)
print("EXTRACTION SUMMARY")
print("=" * 60)

print(f"\n📋 Basic Example Fields Extracted: {len(result_with_reasoning['fields'])}")
for field in result_with_reasoning["fields"]:
    status = "✅ Found" if field["field_value"] else "❌ Not Found"
    confidence = f"({field['field_confidence']:.2f})"
    print(f"  {field['field_name']}: {status} {confidence}")

print(f"\n📋 Complex Example Fields Extracted: {len(complex_result['fields'])}")
for field in complex_result["fields"]:
    status = "✅ Found" if field["field_value"] else "❌ Not Found"
    confidence = f"({field['field_confidence']:.2f})"
    value = (
        field["field_value"][:30] + "..."
        if field["field_value"] and len(field["field_value"]) > 30
        else field["field_value"]
    )
    print(f"  {field['field_name']}: {status} {confidence} - '{value}'")

print("\n" + "=" * 60)
print("✨ All examples completed successfully!")
print("=" * 60)

## Example 4: Conversation Evaluation

Now let's evaluate conversation quality using the `AssertsEvaluator` class:

In [ ]:
# Initialize evaluator with evaluation steps
evaluator = AssertsEvaluator(
    api_key=api_key,
    evaluation_steps=[
        "Did the agent ask for the caller's name?",
        "Did the agent offer to help the caller?",
        "Was the agent polite and professional?",
    ],
    threshold=0.7,
)

# Evaluate conversation
evaluation_conversation = {
    "messages": [
        {"role": "user", "content": "Hi, I need help with my account"},
        {
            "role": "assistant",
            "content": "Hello! I'd be happy to help. May I have your name?",
        },
        {"role": "user", "content": "My name is John Smith"},
        {
            "role": "assistant",
            "content": "Thank you, Mr. Smith. Let me pull up your account information right away.",
        },
    ]
}

print("Evaluating conversation quality...")
eval_result = evaluator.evaluate(evaluation_conversation)
print("\n✅ Evaluation Result:")
print(json.dumps(eval_result, indent=2))

### Evaluation Summary

In [ ]:
print(f"📊 Overall Score: {eval_result['result']['score']:.2f}/10")
print(f"🎯 Confidence: {eval_result['result']['confidence']:.2%}")
print(f"✅ Success: {eval_result['result']['success']}")
print(f"\n💭 Reason: {eval_result['result']['reason']}")

## Next Steps

- 📚 Check out the [documentation](https://github.com/ashishorkalra/automatic-goggles-public#readme)
- 🔍 Try with your own conversation transcripts
- ⚙️ Experiment with custom field definitions
- 💬 Use the `AssertsEvaluator` class for conversation quality evaluation

Happy extracting! 🚀